# PA Opioid Epidemic

In [59]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

In [60]:
overdose_dat = 'Overdose_Information_Network_Data_CY_January_2018_-_Current_Monthly_County_State_Police.csv'
county_geo = 'PaCounty2020_01.geojson'

In [61]:
ovd = pd.read_csv(overdose_dat)

In [62]:
df = ovd.drop(['Incident Time', 'Response Time Desc', 'Revive Action Desc', 'Response Desc', 'Victim County', 'Victim OD Drug ID', 'Third Party Admin Desc', 'Administration ID', 'Dose Count', 'Dose Unit', 'Dose Desc', 'Incident ID', 'Victim State', 'Incident State', 'Victim County Latitude and Longitude', 'Incident County Latitude and Longitude', 'Victim ID', 'Victim County FIPS Code', 'Incident County FIPS Code', 'Victim County Latitude', 'Victim County Longitude'], axis=1)
df = df.rename(columns={"Survive": "Survived", "Ethnicity Desc": "Hispanic", "Accidental Exposure": "Accident", "Age Range": "Age", "Naloxone Administered": "Naloxone", "Incident County Longitude": "Longitude", "Incident County Latitude": "Latitude", "Susp OD Drug Desc": "Drug", "Gender Desc": "Gender", "Incident Date": "Date", "Incident County Name": "County"})

In [63]:
df['Date'] = df['Date'].replace(to_replace=r'\/', value=':', regex=True)
df['Date'] = pd.to_datetime(df['Date'], format='%m:%d:%Y')

In [64]:
df['Age'] = df['Age'].replace(to_replace=r'\s\-\s', value='-', regex=True)
df['Age'] = df['Age'].replace(to_replace=r'14-Oct', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'0-9', value='0-14')
df['Age'] = df['Age'].replace(to_replace=r'70-79', value='70-99')
df['Age'] = df['Age'].replace(to_replace=r'80-*', value='70-99')

In [65]:
df['Gender'] = df['Gender'].replace(to_replace=r'Male', value='M')
df['Gender'] = df['Gender'].replace(to_replace=r'Female', value='F')
df['Gender'] = df['Gender'].replace(to_replace=r'Unknown', value='U')

In [66]:
df['Survived'] = df['Survived'].replace(to_replace=r'Y', value='T')
df['Survived'] = df['Survived'].replace(to_replace=r'N', value='F')

In [67]:
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Not Hispanic', value='F')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Hispanic', value='T')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Unknown', value='F')
df['Hispanic'] = df['Hispanic'].replace(to_replace=r'Mongolian', value='F')

In [68]:
df['Race'] = df['Race'].replace(to_replace=r'Unknown', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Asian or Pacific Islander', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'American Indian or Alaskan Native', value='Other')
df['Race'] = df['Race'].replace(to_replace=r'Other', value='O')
df['Race'] = df['Race'].replace(to_replace=r'White', value='W')
df['Race'] = df['Race'].replace(to_replace=r'Black', value='B')

In [69]:
df['Drug'] = df['Drug'].str.lower()
df['Drug'] = df['Drug'].replace(to_replace=r'benzodiazepines (i.e.valium, xanax, ativan, etc)', value='benzodiazepines')
df['Drug'] = df['Drug'].replace(to_replace=r'buprenorphine (i.e. suboxone, sublocade, subutex, butrans, buprenex, etc)', value='buprenorphine')
df['Drug'] = df['Drug'].replace(to_replace=r'barbiturates (i.e. amytal, nembutal, etc)', value='barbiturates')
df['Drug'] = df['Drug'].replace(to_replace=r'fentanyl analog/other synthetic opioid', value='fentanyl')

In [70]:
df['Accident'] = df['Accident'].replace(to_replace=r'N', value='F')
df['Accident'] = df['Accident'].replace(to_replace=r'Y', value='T')

In [71]:
df['Naloxone'] = df['Naloxone'].replace(to_replace=r'N', value='F')
df['Naloxone'] = df['Naloxone'].replace(to_replace=r'Y', value='T')

In [72]:
df['Survived'] = df['Survived'].replace(to_replace=r'N', value='F')
df['Survived'] = df['Survived'].replace(to_replace=r'Y', value='T')
df['Survived'] = df['Survived'].replace(to_replace=r'U', value='U')

In [73]:
df.head()

,Date,Day,County,Gender,Age,Race,Hispanic,Accident,Drug,Naloxone,Survived,Latitude,Longitude
0,2020-07-02,Thursday,York,M,50-59,W,T,F,heroin,T,T,39.921925,-76.725761
1,2020-09-20,Sunday,York,M,20-24,W,F,F,fentanyl,T,T,39.921925,-76.725761
2,2020-09-11,Friday,Union,M,50-59,W,F,F,heroin,F,F,40.964731,-77.064158
3,2020-07-10,Friday,York,M,40-49,W,F,F,fentanyl,T,T,39.921925,-76.725761
4,2020-06-20,Saturday,York,M,40-49,W,F,F,fentanyl,F,F,39.921925,-76.725761


In [74]:
df_counties = df.groupby('County')['Survived'].transform('sum')

In [75]:
df_counties

0        TTTFTTTFTFTUTTTTTTTTFTFTTTFTTTTTTTTFTTTFTTFTTF...
1        TTTFTTTFTFTUTTTTTTTTFTFTTTFTTTTTTTTFTTTFTTFTTF...
2                                FUTTTTTUFFUFTUTTTFTTTTFTF
3        TTTFTTTFTFTUTTTTTTTTFTFTTTFTTTTTTTTFTTTFTTFTTF...
4        TTTFTTTFTFTUTTTTTTTTFTFTTTFTTTTTTTTFTTTFTTFTTF...
                               ...                        
23703    FUTTTTTTTFUFFTTTTFTTTTTFTTFTTTTTFTTTTTTTTTTTUT...
23704    FTTTTTTTTFTTFTTFTTTTTTTFTFTTUFFTFFFTTTFTTTTTTT...
23705    FFTTTTTFFTTTTFTTUTUFTFTFTTTTFTTFTTFTTTTTTUFFTF...
23706    FFTTTTTFFTTTTFTTUTUFTFTFTTTTFTTFTTFTTTTTTUFFTF...
23707    FFTFTTTTFTTTTTFTTUTFTTTTTFTUTTTTTTFTTTTTTTTTTT...
Name: Survived, Length: 23708, dtype: object

In [ ]:
df.index.values

In [ ]:
latitude = 40.903323 
longitude = -77.594527
pa = folium.Map(location=[latitude, longitude], zoom_start=8)

folium.GeoJson(county_geo, name="geojson").add_to(pa)

folium.LayerControl().add_to(pa)

pa

In [ ]:
latitude = 41.203323 
longitude = -77.194527
m = folium.Map(location=[latitude, longitude], zoom_start=7)




m